In [17]:
import os
import csv
import pandas as pd

import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

#Parse Sysbench IO Results

In [18]:
docker_async_df = pd.DataFrame(columns=['Number of Threads',
                           'Number of Files',
                           'File Size',
                           'Total File Size',
                           'Block Size',
                           'Fsync Periodicty',
                           'I/O Mode (sync/async)', 
                           'Access Mode (seq/rnd)',
                           'Read/Write',
                           'File Operations (reads/s)',
                           'File Operations (writes/s)',
                           'File Operations (fsyncs/s)',
                           'Read Throughput (MiB/s)',
                           'Write Throughput (MiB/s)',
                           'Total Time (s)', 
                           'Total Number of Events',
                           'Min Latency (ms)',
                           'Avg Latency (ms)',
                           'Max Latency (ms)',
                           '95th Percentile Latency (ms)',
                           'Total Execution Time across Threads (s)',
                           'Avg Number of Events per Thread',
                           'Avg Execution Time per Thread (s)']) 
                           

In [19]:
cols = docker_async_df.columns

In [20]:
def readFromOutputs(filepath, df):
    for outputFile in os.listdir(filepath):
        if outputFile == '.ipynb_checkpoints':
            continue
        if outputFile == '.gitkeep':
            continue
        row = {}
        row['Number of Threads'] = int(outputFile.split("_")[1])
        row['Access Mode (seq/rnd)'] = 'Sequential' if 'seq' in outputFile.split("_")[2] else 'Random'
        row['Read/Write'] = 'Read' if 'rd' in outputFile.split("_")[2] else 'Write'
        col_no = 9 

        full_filepath = os.path.join(filepath, outputFile)
        
        
        with open(full_filepath, 'r') as f:
            
            for line in f:
                if 'files' in line:
                    row['Number of Files'] = (line.split()[0])

                if 'files' in line:
                    row['File Size'] = (line.split()[2])
      
                if 'total file size' in line: 
                    row['Total File Size'] = (line.split()[0])
                    
                if 'Block size' in line:
                    row['Block Size'] = (line.split()[2])
                    
                if 'Periodic FSYNC' in line: 
                    row['Fsync Periodicty'] = int(line.split()[6])

                if 'I/O mode' in line: 
                    row['I/O Mode (sync/async)'] = line.split()[1]
                    
                if ':' in line and ('WARNING' not in line) and ('threads' not in line) and ('flags' not in line) and ('options' not in line) and('IO requests' not in line) and ('random IO test' not in line):
                    result = line.rsplit(':', 1)[1].strip()
                    if result != None:
                        if 's' in result:
                            row[cols[col_no]] = float(result[:-1])
                            col_no += 1
                        elif '/' in result:
                            row[cols[col_no]] = float(result.split('/')[0])
                            col_no += 1
                        elif result != '':
                            row[cols[col_no]] = float(result)
                            col_no += 1
                    
        # print("ROW", row, "\n")
        df = df.append(row, ignore_index=True)
            
    return df

In [21]:
docker_async_df = readFromOutputs("dockerOutputs_Async/", docker_async_df)

/tmp/ipykernel_1015715/648799220.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_1015715/648799220.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_1015715/648799220.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_1015715/648799220.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_1015715/648799220.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_1015715/648799220.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas

In [22]:
docker_async_df.to_csv('dockerAsyncFileIO.csv')

In [23]:
docker_async_df['SeqRndWriteRead'] = docker_async_df['Access Mode (seq/rnd)'] + docker_async_df['Read/Write']

In [24]:
docker_async_df = docker_async_df.sort_values(by=['Number of Threads'])

In [25]:
import plotly.express as px
fig = px.line(docker_async_df, x='Number of Threads', y='Total Number of Events', color='SeqRndWriteRead', markers=True, title=f"Total Number of Events in 30s for Docker Container File IO Async Read/Write. Total File Size:{docker_async_df['Total File Size'][0]}, No. of files: {docker_async_df['Number of Files'][0]}.")
fig.show()

##Baremetal Sync Outputs

In [26]:
docker_sync_df = pd.DataFrame(columns=['Number of Threads',
                           'Number of Files',
                           'File Size',
                           'Total File Size',
                           'Block Size',
                           'Fsync Periodicty',
                           'I/O Mode (sync/async)', 
                           'Access Mode (seq/rnd)',
                           'Read/Write',
                           'File Operations (reads/s)',
                           'File Operations (writes/s)',
                           'File Operations (fsyncs/s)',
                           'Read Throughput (MiB/s)',
                           'Write Throughput (MiB/s)',
                           'Total Time (s)', 
                           'Total Number of Events',
                           'Min Latency (ms)',
                           'Avg Latency (ms)',
                           'Max Latency (ms)',
                           '95th Percentile Latency (ms)',
                           'Total Execution Time across Threads (s)',
                           'Avg Number of Events per Thread',
                           'Avg Execution Time per Thread (s)'])

In [27]:
cols = docker_sync_df.columns

In [28]:
docker_sync_df = readFromOutputs("dockerOutputs_Sync/", docker_sync_df)

/tmp/ipykernel_1015715/648799220.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_1015715/648799220.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_1015715/648799220.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_1015715/648799220.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_1015715/648799220.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_1015715/648799220.py:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas

In [29]:
docker_sync_df.to_csv('dockerSyncFileIO.csv')

In [30]:
docker_sync_df['SeqRndWriteRead'] = docker_sync_df['Access Mode (seq/rnd)'] + docker_sync_df['Read/Write']

In [31]:
docker_sync_df = docker_sync_df.sort_values(by=['Number of Threads'])

In [32]:
import plotly.express as px
fig = px.line(docker_sync_df, x='Number of Threads', y='Total Number of Events', color='SeqRndWriteRead', markers=True, title=f"Total Number of Events in 30s for Docker Container File IO Sync Read/Write. Total File Size:{docker_sync_df['Total File Size'][0]}, No. of files: {docker_sync_df['Number of Files'][0]}.")
fig.show()